In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ResourceClosedError, ProgrammingError
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from lxml import etree

# necessary to allow import of local modules
cwd = os.getcwd()
sys.path.append(cwd)

from connection import Connection

In [ ]:
from connection import Connection

db = Connection("postgresql+psycopg://ying.tan@localhost/osm_db")



In [ ]:
db.get_tables()

In [ ]:
db.execute("select * from nyc_census_tracts limit 100")

In [ ]:

sql = """
with transformed_baskets as (
    select b.id as basket_id, st_transform(b.geom, 2263) as basket_geom, 
    (select nct.gid from nyc_census_tracts nct where st_contains(nct.geom, st_transform(b.geom, 2263))) as gid 
    from baskets b

)
select t.gid, t.ntaname, count(basket_id) as num_baskets, ST_NumGeometries(t.geom), t.geom from nyc_census_tracts t
left join transformed_baskets b on t.gid = b.gid
group by t.gid order by t.gid asc;
"""


print("spot check that the number of baskets and polygons per census tract look correct.")
nct = gpd.GeoDataFrame.from_postgis(sql, db.engine)
nct.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
nct.plot('st_numgeometries', ax=ax, legend=True)
plt.title("number of gemoetries in each census tract")

In [ ]:


# plot number of baskets in each census tract
baskets = gpd.GeoDataFrame.from_postgis("select id, type, lat, lng, st_asbinary(st_transform(geom, 2263)) as geom from baskets", db.engine)
baskets.geom = baskets.geom.set_crs("EPSG:2263")
fig, ax = plt.subplots(figsize=(10, 10))
# ymin, ymax = 215000, 212000
# xmin, xmax = 985000, 987500
# ax.set_xlim([xmin, xmax])
# ax.set_ylim([ymin, ymax])
nct.plot('count', ax=ax, legend=True)
baskets.plot(ax=ax, markersize=0.5)

plt.title("number of litter baskets in each census tract")

In [ ]:

# Leaflet does not support 2263, so convert all geos to 4326
baskets4326 = baskets.geom.to_crs("EPSG:4326")
tracts4326 = gpd.GeoDataFrame({ 'gid': nct['gid'], 'geom': nct.geom.to_crs(crs=4326), 'count': nct['count'], 'cdtaname': nct['cdtaname'] })
tracts4326.set_geometry('geom', crs="EPSG:4326", inplace=True)
loc = list(baskets4326.get_coordinates().iloc[0])
loc = [40.7237415632241,-73.97571566678945]
# loc = [40.730610, -73.935242]
map = folium.Map(location=loc, tiles="CartoDB Positron", zoom_start=14)


for index, row in tracts4326.iterrows():
    geo_j = gpd.GeoSeries(row['geom']).to_json()
    c = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    folium.Popup(f"gid: {row['gid']}\ncount : {row['count']}").add_to(c)
    c.add_to(map)

for index, row in baskets4326.get_coordinates().iterrows():
    c = folium.vector_layers.CircleMarker(
        [row['y'], row['x']], radius=0.5
    )
    map.add_child(c)

# This map can be used to spotcheck that calculations are done correctly
map


In [ ]:
# --select (ST_SquareGrid(0.001, ST_Transform(geom,4326))).* from nyc_census_tracts where gid = 1;
# Testing: Return a grid of centroids that cover the census tract with gid = 1
ellis_island_gid = 1

sql = f"""
select st_centroid((st_squaregrid(0.001, st_transform(t.geom,4326))).geom) as geom from nyc_census_tracts t where gid = {ellis_island_gid}
--select st_centroid((st_squaregrid(1000, st_transform(t.geom,2263))).geom) as geom from nyc_census_tracts t where gid = {ellis_island_gid}
"""
grids = gpd.GeoDataFrame.from_postgis(sql, db.engine)


In [ ]:
# and map it
center = nct.loc[nct.gid == ellis_island_gid].centroid.to_crs("EPSG:4326")[0]

loc = [center.y, center.x]
map = folium.Map(location=loc, tiles="CartoDB Positron", zoom_start=14, scrollWheelZoom=False)
for index, row in grids.geom.get_coordinates().iterrows():
    c = folium.vector_layers.CircleMarker(
        [row['y'], row['x']], radius=0.5
    )
    map.add_child(c)
    
print("preview grid generation that covers a particular census tract")
map

In [ ]:
# sanity check for finding the bounds of nyc
sql = """
select st_transform(st_setsrid(st_extent(x.geom), 2263), 4326) as geom from 
(select st_union(t.geom) as geom from nyc_census_tracts t) as x;
"""
nyc_extent = gpd.GeoDataFrame.from_postgis(sql, db.engine)
nyc_extent

map = folium.Map(location=loc, tiles="CartoDB Positron", zoom_start=9)
geo_j = nyc_extent.geom.to_json()
c = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})

c.add_to(map)


fig, ax = plt.subplots(figsize=(10, 10))
tracts4326.plot('count', ax=ax, legend=True)
nyc_extent.plot(ax=ax, alpha=0.3)
xmin, ymin, xmax, ymax = nyc_extent.total_bounds
print("bounds of nyc, which can be used to query overpass turbo", [ymin, xmin, ymax, xmax])

In [ ]:
from urllib.parse import quote_plus

query = """
[out:xml][timeout:25];

(
  way["highway"="footway"](40.49600922391659, -74.25715844001557, 40.91528149796315, -73.69921524007877);
  way["footway"="sidewalk"](40.49600922391659, -74.25715844001557, 40.91528149796315, -73.69921524007877);
);

// print results
(
  ._; 
  >>;
);

out;

"""

arg = quote_plus(query)
url = '"' + 'https://overpass-api.de/api/interpreter?data=' + arg + '"'

OVERPASS_FILENAME = "nyc_footpaths_only_recurse_down.osm"

# This command downloads the paths 
!curl -o {OVERPASS_FILENAME} {url}

In [ ]:
# The resulting file needs to be modified so all untagged nodes are given a tag - otherwise they won't be
# imported into the db below.

RESULTS_FILENAME = f"parsed.osm"

tree = etree.parse(OVERPASS_FILENAME)

for n in tree.iter("node"):
    # add a tag to untagged nodes so that they get parsed
    if len(n) == 0: 
        n.append(etree.Element("tag", k="foo", v="bar"))

tree.write(RESULTS_FILENAME)



                

In [ ]:

# For safety we drop the previous tables first
sql = """
-- these are used in the default import
drop table if exists osm_footpaths_point;
drop table if exists osm_footpaths_line;
drop table if exists osm_footpaths_polygon;
drop table if exists osm_footpaths_roads;

-- these are used in the lua style import
drop table if exists nodes;
drop table if exists ways;
"""

db.raw_execute(sql)

def run_default_import():
    # Now run the tool to import data into the postgres db:
    # Note that nyc_footpaths_only.style is a style file copied from:
    # /opt/homebrew/opt/osm2pgsql/share/osm2pgsql/default.style
    # It tells osm2pgsql how to import data into postgres
    !osm2pgsql -c -d osm_db -H localhost --prefix osm_footpaths -S nyc_footpaths_only.style "$(pwd)/{RESULTS_FILENAME}"

def run_lua_import():
    !osm2pgsql -c -d osm_db -H localhost --output=flex --style=simple_import.lua "$(pwd)/{RESULTS_FILENAME}"

run_lua_import()
    

In [ ]:
# get all the roads
sql = """ 
 select l.way as geom, highway, footway
 from osm_footpaths_line l
"""

ways = gpd.GeoDataFrame.from_postgis(sql, db.engine)

fig, ax = plt.subplots(figsize=(10, 10))
ways.plot('highway', ax=ax, legend=True)
ways.plot('footway', ax=ax, legend=True)


In [ ]:
function_sql = """
CREATE OR REPLACE FUNCTION testing(s_node integer)
RETURNS integer AS $$

declare
s int8 = 3;
my_array INTEGER[] := '{1, 2, 3, 4, 5}';
begin
    my_array[10] = 10;
    return my_array[10];
end;
$$
language plpgsql;
"""

db.raw_execute(function_sql)

db.execute("""
select testing(1);
""")




In [ ]:
# Insert the exact coordinates of the nodes, create edges, and record their distances
db.raw_execute("""
with temp_node1 as (
    select e.id, n.node_id, n.geom
    from nodes n
    inner join edges e on e.node1 = n.node_id
),
temp_node2 as (
    select e.id, n.node_id, n.geom
    from nodes n
    inner join edges e on e.node2 = n.node_id
),

temp_joined as (
    select 
        t1.id as id,
        t1.geom as node1,
        t2.geom as node2,
        st_distance(t1.geom, t2.geom) as distance,
        st_makeline(t1.geom, t2.geom) as way
    from
        temp_node1 as t1
    inner join temp_node2 t2 on t1.id = t2.id
)

update edges
    set 
        node1_point = t.node1,
        node2_point = t.node2,
        distance = t.distance,
        way = t.way
    from
        temp_joined t
    where t.id = edges.id;
""")



In [ ]:
# This sql finds the closest point on each way for every node, and checks that every basket
# has a closest point on an edge.
sql = """
with

basket_intersections as (
select
    d.id, e.id as edge_id,
    e.node1_point,
    e.node2_point,
    st_astext(d.geom) as basket,
    st_closestpoint(e.way, d.geom) as closest_point,
    st_distance(st_closestpoint(e.way, d.geom), d.geom) as distance,
    row_number() over (partition by d.id order by st_distance(st_closestpoint(e.way, d.geom), d.geom)) as rownum
    from baskets d
    left join edges e on st_dwithin(d.geom, e.way , 50)
)
select
    st_dwithin(e.way, b.closest_point, 1), count(1)
from basket_intersections b
join edges e on b.edge_id = e.id group by st_dwithin(e.way, b.closest_point, 1)
limit 10;
"""
db.execute(sql)